In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

In [3]:
def generate_dummy_chess_data(num_samples):
    """
    Generate a dummy dataset for chess move prediction.

    Args:
        num_samples (int): Number of samples to generate.

    Returns:
        X (np.ndarray): Input features (board states) of shape (num_samples, 8, 8, 12).
        y (np.ndarray): Target labels (moves) of shape (num_samples, 1).
    """
    # Initialize empty arrays for board states (X) and target moves (y)
    X = np.zeros((num_samples, 8, 8, 12), dtype=np.float32)
    y = np.zeros((num_samples, 1), dtype=np.int64)

    for i in range(num_samples):
        # Randomly generate a board state
        for row in range(8):
            for col in range(8):
                # Randomly select a piece type (0 to 11 for 12 piece types)
                piece_type = random.randint(0, 11)
                # Set the corresponding channel to 1 to represent the piece presence
                X[i, row, col, piece_type] = 1

        # Randomly select a move (0 to 63 for 64 board positions)
        y[i] = random.randint(0, 63)

    return X, y

# Prepare the dataset
num_samples = 1000  # Number of samples to generate
X_train, y_train = generate_dummy_chess_data(num_samples)

# Print the shapes of the generated dataset
print(f"Generated dataset shapes: X_train = {X_train.shape}, y_train = {y_train.shape}")

NameError: name 'np' is not defined

In [ ]:
X_train = np.random.rand(1000, 8, 8, 12)  # 1000 board states, 8x8 board, 12 piece types
y_train = np.random.randint(0, 64, (1000, 1))  # 1000 moves (0-63 for board positions)

# Convert numpy arrays to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long).squeeze()

In [ ]:
class ChessMovePredictor(nn.Module):
    def __init__(self):
        super(ChessMovePredictor, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=12, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # Flattened feature size
        self.fc2 = nn.Linear(128, 64)  # Output layer for 64 possible board positions

    def forward(self, x):
        # Forward pass through the network
        x = torch.relu(self.conv1(x))  # Apply ReLU activation after the first conv layer
        x = torch.relu(self.conv2(x))  # Apply ReLU activation after the second conv layer
        
        # Flatten the output from the conv layers to feed into fully connected layers
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))  # Apply ReLU activation after the first fully connected layer
        x = self.fc2(x)  # Output layer (logits for classification)
        
        return x

In [ ]:
# Initialize the model, loss function, and optimizer
model = ChessMovePredictor()
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [ ]:
# Training parameters
num_epochs = 10
batch_size = 32

# Training loop
for epoch in range(num_epochs):
    permutation = torch.randperm(X_train.size()[0])  # Shuffle the training data
    
    for i in range(0, X_train.size()[0], batch_size):
        indices = permutation[i:i + batch_size]  # Get batch indices
        batch_x, batch_y = X_train[indices], y_train[indices]  # Get the batch data
        
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(batch_x)  # Forward pass
        loss = criterion(outputs, batch_y)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')  # Print loss after each epoch

In [ ]:
# Evaluate the model (simple accuracy for demonstration purposes)
with torch.no_grad():  # Disable gradient calculation
    outputs = model(X_train)  # Get model predictions
    _, predicted = torch.max(outputs, 1)  # Get the index of the highest logit
    accuracy = (predicted == y_train).float().mean()  # Calculate accuracy
    print(f'Accuracy: {accuracy.item():.4f}')  # Print accuracy